In [1]:
import warnings
warnings.filterwarnings('ignore')
#warnings.filterwarnings(action='once')

In [2]:
import numpy as np
import numpy.testing as npt
#from fixtures import dtlz2_5x_3f_data_based  # noqa: F401


import polars as pl

import pandas as pd


from desdeo.mcdm.nautili import (
    calculate_distance_to_front,
    calculate_navigation_point,
    solve_reachable_bounds,
    solve_reachable_solution,
    agg_maxmin, 
    agg_maxmin_cones,
    nautili_all_steps,
    NAUTILI_Response, 
    aggregate,
    nautili_all_steps,
    nautili_init
)

from desdeo.problem import (
    binh_and_korn,
    objective_dict_to_numpy_array,
    river_pollution_problem,
    get_nadir_dict,
    get_ideal_dict,
    dtlz2,
    #variable_dict_to_numpy_array,
    objective_dict_to_numpy_array
)
from desdeo.tools import (
    BonminOptions,
    NevergradGenericOptions,
    NevergradGenericSolver,
    PyomoBonminSolver,
    ScipyMinimizeSolver,
)


#def test_dtzl2(pref_agg_method):
    #problem = river_pollution_problem
problem = dtlz2(10,3)
nadir = get_nadir_dict(problem)
ideal = get_ideal_dict(problem)
total_steps = 3
pref_agg_method="maxmin"

initial_response = nautili_init(problem)
rps = {
    "DM1": {"f_1": 0.8, "f_2": 0.7, "f_3": 0.6},
    "DM2": {"f_1": 0.7, "f_2": 0.8, "f_3": 0.5},
    "DM3": {"f_1": 0.5, "f_2": 0.6, "f_3": 0.8},
    #"DM4": {"f_1": 0.5, "f_2": 0.6, "f_3": 0.8},
}
all_resp = nautili_all_steps(
    problem,
    total_steps,
    rps,
    [initial_response], # Note that this is a list of NAUTILUS_Response objects
    pref_agg_method=pref_agg_method, # used pref agg method
)
print(all_resp)
print(all_resp[-1].reference_points)
print(all_resp[-1].group_improvement_direction)
print(all_resp[-1].navigation_point)

#def solve_prob(problem, pref_agg_method):

nadir = get_nadir_dict(problem)
ideal = get_ideal_dict(problem)
rps = {
    "DM1": {"f_1": 0.8, "f_2": 0.7, "f_3": 0.6},
    "DM2": {"f_1": 0.7, "f_2": 0.8, "f_3": 0.5},
    "DM3": {"f_1": 0.5, "f_2": 0.6, "f_3": 0.8},
    #"DM4": {"f_1": 0.5, "f_2": 0.6, "f_3": 0.8},
}
initial_response = nautili_init(problem)
all_resp = nautili_all_steps(problem, 3, rps, [initial_response], pref_agg_method)
#return all_resp




maxmin
boiunds (0.0, 2.0)
boiunds (0.0, 2.0)
boiunds (0.0, 2.0)
boiunds (0.0, 2.0)
`xtol` termination condition is satisfied.
Number of iterations: 336, function evaluations: 1680, CG iterations: 0, optimality: 1.96e-01, constraint violation: 1.00e+00, execution time: 0.26 s.
[8.00700745e-09 8.00738320e-09 8.00745991e-09]
[NAUTILI_Response(step_number=1, distance_to_front=33.33333333333335, reference_points={'DM1': {'f_1': 0.8, 'f_2': 0.7, 'f_3': 0.6}, 'DM2': {'f_1': 0.7, 'f_2': 0.8, 'f_3': 0.5}, 'DM3': {'f_1': 0.5, 'f_2': 0.6, 'f_3': 0.8}}, improvement_directions={'DM1': array([1.2, 1.3, 1.4]), 'DM2': array([1.3, 1.2, 1.5]), 'DM3': array([1.5, 1.4, 1.2])}, group_improvement_direction={'f_1': 1.9999999919929925, 'f_2': 1.9999999919926168, 'f_3': 1.9999999919925402}, navigation_point={'f_1': 1.5364709777317986, 'f_2': 1.5364973879863197, 'f_3': 1.5354603644147513}, reachable_solution={'f_1': 0.6094129331953962, 'f_2': 0.6094921639589597, 'f_3': 0.6063810932442545}, reachable_bounds={'lo

In [4]:
#!pip install odfpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.0/717.0 kB 4.9 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for odfpy: filename=odfpy-1.4.1-py2.py3-none-any.whl size=160672 sha256=cf111875043c6b6d4498173a574554de7f81e6b204847929d88e84bba5c283d8
  Stored in directory: /home/jp/.cache/pip/wheels/36/5d/63/8243a7ee78fff0f944d638fd0e66d7278888f5e2285d7346b6
Successfully built odfpy


In [5]:
# THIs is what I want!
ff = pd.read_excel("results/test.ods")
ff

,Type,f_1,f_2,f_3,Iteration
0,DM1,0.7,0.5,0.7,0
1,DM2,0.6,0.5,0.6,0
2,DM3,0.5,0.8,0.5,0
3,NAVPOINT,1.0,1.0,1.0,0
4,GID,0.5,0.5,0.5,0
5,DM1,0.7,0.5,0.7,1
6,DM2,0.6,0.5,0.6,1
7,DM3,0.5,0.8,0.5,1
8,NAVPOINT,0.8,0.8,0.8,1
9,GID,0.5,0.5,0.5,1


In [41]:
cols = ["DM1", "DM2", "DM3"]

In [42]:
lower_bounds = pd.DataFrame(
    [response.reachable_bounds["lower_bounds"] for response in all_resp]
)
upper_bounds = pd.DataFrame(
    [response.reachable_bounds["upper_bounds"] for response in all_resp]
)
reference_points = pd.DataFrame(
    [response.reference_points for response in all_resp[1:]]
)
navigation_points = pd.DataFrame(
    [response.navigation_point for response in all_resp]
)
reachable_sols = pd.DataFrame(
    [response.reachable_solution for response in all_resp]
)
group_dirs = pd.DataFrame(
    [all_resp[0].group_improvement_direction]
)
steps = pd.DataFrame([response.step_number for response in all_resp])

In [50]:
id_dict = {
    "step_n": [response.step_number for response in all_resp],
    "DM1": [response.reference_points["DM1"] for response in all_resp],
    "DM2": [response.reference_points["DM2"] for response in all_resp],
}
id_dict

{'step_n': [1, 2, 3],
 'DM1': [{'f_1': 0.8, 'f_2': 0.7, 'f_3': 0.6},
  {'f_1': 0.8, 'f_2': 0.7, 'f_3': 0.6},
  {'f_1': 0.8, 'f_2': 0.7, 'f_3': 0.6}],
 'DM2': [{'f_1': 0.7, 'f_2': 0.8, 'f_3': 0.5},
  {'f_1': 0.7, 'f_2': 0.8, 'f_3': 0.5},
  {'f_1': 0.7, 'f_2': 0.8, 'f_3': 0.5}]}

In [65]:
all_resp

[NAUTILI_Response(step_number=1, distance_to_front=33.33333333333335, reference_points={'DM1': {'f_1': 0.8, 'f_2': 0.7, 'f_3': 0.6}, 'DM2': {'f_1': 0.7, 'f_2': 0.8, 'f_3': 0.5}, 'DM3': {'f_1': 0.5, 'f_2': 0.6, 'f_3': 0.8}}, improvement_directions={'DM1': array([1.2, 1.3, 1.4]), 'DM2': array([1.3, 1.2, 1.5]), 'DM3': array([1.5, 1.4, 1.2])}, group_improvement_direction={'f_1': 1.9999999919929925, 'f_2': 1.9999999919926168, 'f_3': 1.9999999919925402}, navigation_point={'f_1': 1.5364709777317986, 'f_2': 1.5364973879863197, 'f_3': 1.5354603644147513}, reachable_solution={'f_1': 0.6094129331953962, 'f_2': 0.6094921639589597, 'f_3': 0.6063810932442545}, reachable_bounds={'lower_bounds': {'f_1': 3.90654895990358e-10, 'f_2': 0.0, 'f_3': 1.3470386024085117e-05}, 'upper_bounds': {'f_1': 1.5364709777317986, 'f_2': 1.5364973879863197, 'f_3': 1.5354603644147513}}),
 NAUTILI_Response(step_number=2, distance_to_front=66.66666666666667, reference_points={'DM1': {'f_1': 0.8, 'f_2': 0.7, 'f_3': 0.6}, 'DM

In [120]:
resp0 = all_resp[0]
resp0

NAUTILI_Response(step_number=1, distance_to_front=33.33333333333335, reference_points={'DM1': {'f_1': 0.8, 'f_2': 0.7, 'f_3': 0.6}, 'DM2': {'f_1': 0.7, 'f_2': 0.8, 'f_3': 0.5}, 'DM3': {'f_1': 0.5, 'f_2': 0.6, 'f_3': 0.8}}, improvement_directions={'DM1': array([1.2, 1.3, 1.4]), 'DM2': array([1.3, 1.2, 1.5]), 'DM3': array([1.5, 1.4, 1.2])}, group_improvement_direction={'f_1': 1.9999999919929925, 'f_2': 1.9999999919926168, 'f_3': 1.9999999919925402}, navigation_point={'f_1': 1.5364709777317986, 'f_2': 1.5364973879863197, 'f_3': 1.5354603644147513}, reachable_solution={'f_1': 0.6094129331953962, 'f_2': 0.6094921639589597, 'f_3': 0.6063810932442545}, reachable_bounds={'lower_bounds': {'f_1': 3.90654895990358e-10, 'f_2': 0.0, 'f_3': 1.3470386024085117e-05}, 'upper_bounds': {'f_1': 1.5364709777317986, 'f_2': 1.5364973879863197, 'f_3': 1.5354603644147513}})

In [130]:
df_resp0 = pd.DataFrame(resp0)
df_resp0

,0,1
0,step_number,1
1,distance_to_front,33.333333
2,reference_points,"{'DM1': {'f_1': 0.8, 'f_2': 0.7, 'f_3': 0.6}, ..."
3,improvement_directions,"{'DM1': [1.2, 1.3, 1.4], 'DM2': [1.3, 1.2, 1.5..."
4,group_improvement_direction,"{'f_1': 1.9999999919929925, 'f_2': 1.999999991..."
5,navigation_point,"{'f_1': 1.5364709777317986, 'f_2': 1.536497387..."
6,reachable_solution,"{'f_1': 0.6094129331953962, 'f_2': 0.609492163..."
7,reachable_bounds,"{'lower_bounds': {'f_1': 3.90654895990358e-10,..."


In [131]:
# cleaned resp0
df_resp0 = df_resp0.drop(index=[1,3,7])
df_resp0

,0,1
0,step_number,1
2,reference_points,"{'DM1': {'f_1': 0.8, 'f_2': 0.7, 'f_3': 0.6}, ..."
4,group_improvement_direction,"{'f_1': 1.9999999919929925, 'f_2': 1.999999991..."
5,navigation_point,"{'f_1': 1.5364709777317986, 'f_2': 1.536497387..."
6,reachable_solution,"{'f_1': 0.6094129331953962, 'f_2': 0.609492163..."


In [139]:
resp_dict = {
    "step_number": resp0.step_number,
    #"rps": resp0.reference_points,
    "gid": resp0.group_improvement_direction,
    "navpoint": resp0.navigation_point,
    "reachable_sol": resp0.reachable_solution
}
resp_dict

{'step_number': 1,
 'gid': {'f_1': 1.9999999919929925,
  'f_2': 1.9999999919926168,
  'f_3': 1.9999999919925402},
 'navpoint': {'f_1': 1.5364709777317986,
  'f_2': 1.5364973879863197,
  'f_3': 1.5354603644147513},
 'reachable_sol': {'f_1': 0.6094129331953962,
  'f_2': 0.6094921639589597,
  'f_3': 0.6063810932442545}}

In [140]:
resp0.reference_points

{'DM1': {'f_1': 0.8, 'f_2': 0.7, 'f_3': 0.6},
 'DM2': {'f_1': 0.7, 'f_2': 0.8, 'f_3': 0.5},
 'DM3': {'f_1': 0.5, 'f_2': 0.6, 'f_3': 0.8}}

In [170]:
resp_prefs = {
    #"step_number": resp0.step_number,
    "DM1": {resp0.reference_points["DM1"],"step_number": resp0.step_number, },
    #"DM2":resp0.reference_points["DM2"],
    #"DM3":resp0.reference_points["DM3"],
    #"DM1": [resp0.reference_points[dm]["f_1"] for dm in resp0.reference_points if "DM1" in cols ],
    #"DM2": [resp0.reference_points[dm] for dm in resp0.reference_points if "DM2" in cols ],
    #"DM3": [resp0.reference_points[dm] for dm in resp0.reference_points if "DM3" in cols ],
}
resp_prefs 

SyntaxError: invalid syntax (1355953853.py, line 3)

In [162]:
idf = pd.DataFrame(resp_prefs,
                #index=[1,2,3],
                #columns=["f_1","f_2", "f_3"]
                )
idf

,step_number,DM1,DM2,DM3
f_1,1,0.8,0.7,0.5
f_2,1,0.7,0.8,0.6
f_3,1,0.6,0.5,0.8


In [169]:
idf2 = idf.transpose()#.set_index("f_1")
#idf2.columns.name("objs")
idf2

,f_1,f_2,f_3
step_number,1.0,1.0,1.0
DM1,0.8,0.7,0.6
DM2,0.7,0.8,0.5
DM3,0.5,0.6,0.8


In [163]:
idf.transpose()

,f_1,f_2,f_3
step_number,1.0,1.0,1.0
DM1,0.8,0.7,0.6
DM2,0.7,0.8,0.5
DM3,0.5,0.6,0.8


In [132]:
id_dict = {
    "step_n": [response.step_number for response in all_resp],
    "f_1": [response.reference_points[dm]["f_1"] for response in all_resp],
    #"f_2": [response.reference_points[]["f_2"] for response in all_resp] ,
    #"f_3": [response.reference_points["DM"]["f_3"] for response in all_resp] ,
        #"DM1": [response.reference_points["DM1"] for response in all_resp],
    #"DM2": [response.reference_points["DM2"] for response in all_resp],
}
id_dict

ValueError: too many values to unpack (expected 2)

In [56]:
idf = pd.DataFrame(id_dict,
                index=[1,2,3],
                columns=["f_1","f_2", "f_3", "DM"]
                )
idf

,f_1,f_2,f_3,DM
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN


In [43]:

rpdict = [response.reference_points for response in all_resp[1:]]

rpdict

[{'DM1': {'f_1': 0.8, 'f_2': 0.7, 'f_3': 0.6},
  'DM2': {'f_1': 0.7, 'f_2': 0.8, 'f_3': 0.5},
  'DM3': {'f_1': 0.5, 'f_2': 0.6, 'f_3': 0.8}},
 {'DM1': {'f_1': 0.8, 'f_2': 0.7, 'f_3': 0.6},
  'DM2': {'f_1': 0.7, 'f_2': 0.8, 'f_3': 0.5},
  'DM3': {'f_1': 0.5, 'f_2': 0.6, 'f_3': 0.8}}]

In [39]:
[response.step_number for response in all_resp]

[1, 2, 3, 4, 5]

In [28]:
reference_points.axes

[RangeIndex(start=0, stop=4, step=1),
 Index(['DM1', 'DM2', 'DM3'], dtype='object')]

In [26]:
[response.reference_points["DM1"] for response in all_resp[1:]]

[{'f_1': 0.7, 'f_2': 0.8, 'f_3': 0.5},
 {'f_1': 0.7, 'f_2': 0.8, 'f_3': 0.5},
 {'f_1': 0.7, 'f_2': 0.8, 'f_3': 0.5},
 {'f_1': 0.7, 'f_2': 0.8, 'f_3': 0.5}]

In [38]:
bigDF = pd.DataFrame(
    [response.step_number for response in all_resp], 
    #[response.step_number for response in all_resp],
    #[response.step_number for response in all_resp],
    #[response.navigation_point for response in all_resp],
    rpdict,
    #[response.reference_points["DM1"] for response in all_resp[1:]],
    #index=cols,
    #columns=cols
    )
bigDF

ValueError: Shape of passed values is (5, 1), indices imply (4, 1)

In [14]:
reference_points

,DM1,DM2,DM3
0,"{'f_1': 0.8, 'f_2': 0.7, 'f_3': 0.6}","{'f_1': 0.7, 'f_2': 0.8, 'f_3': 0.5}","{'f_1': 0.5, 'f_2': 0.6, 'f_3': 0.8}"
1,"{'f_1': 0.8, 'f_2': 0.7, 'f_3': 0.6}","{'f_1': 0.7, 'f_2': 0.8, 'f_3': 0.5}","{'f_1': 0.5, 'f_2': 0.6, 'f_3': 0.8}"
2,"{'f_1': 0.8, 'f_2': 0.7, 'f_3': 0.6}","{'f_1': 0.7, 'f_2': 0.8, 'f_3': 0.5}","{'f_1': 0.5, 'f_2': 0.6, 'f_3': 0.8}"
3,"{'f_1': 0.8, 'f_2': 0.7, 'f_3': 0.6}","{'f_1': 0.7, 'f_2': 0.8, 'f_3': 0.5}","{'f_1': 0.5, 'f_2': 0.6, 'f_3': 0.8}"


In [13]:
steps

,0
0,1
1,2
2,3
3,4
4,5


In [15]:
fs = steps.merge(reference_points)
fs

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [102]:
# what I would like the dataframe to be:
objs = ["f_1", "f_2", "f_3", "DM"]
ideal_dataframe = {
        #"iter": {
        "DM": {"DM1": {
        "f_1": [0.8, 0.8, 0.8],
        "f_2": [0.7,0.7, 0.7],
        "f_3": [0.6,0.6,0.6],
        #"DM": "DM1"
        },
        }
        #"it": { 
        #"DM1": {"f_1": 0.8, "f_2": 0.7, "f_3": 0.6},
        #"DM2": {"f_1": 0.7, "f_2": 0.8, "f_3": 0.5},
        #"DM3": {"f_1": 0.5, "f_2": 0.6, "f_3": 0.8}, 
        #}
        }
ideal_dataframe


{'DM': {'DM1': {'f_1': [0.8, 0.8, 0.8],
   'f_2': [0.7, 0.7, 0.7],
   'f_3': [0.6, 0.6, 0.6]}}}

In [ ]:
"one": pd.Series([1.0, 2.0, 3.0], index=["a", "b", "c"]),

In [109]:
idealdff = {
        #"iter": {
        #"DM": {"DM1": {
        "f_1": pd.Series([0.8, 0.8, 0.8], index=[1,2,3], name="DM1"), # index on nyt step number..
        "f_2": pd.Series([0.7,0.7, 0.7], index=[1,2,3],name="DM1"),
        "f_3": pd.Series([0.6,0.6,0.6], index=[1,2,3],name="DM1"),
}
idealdff

{'f_1': 1    0.8
 2    0.8
 3    0.8
 Name: DM1, dtype: float64,
 'f_2': 1    0.7
 2    0.7
 3    0.7
 Name: DM1, dtype: float64,
 'f_3': 1    0.6
 2    0.6
 3    0.6
 Name: DM1, dtype: float64}

In [119]:
idealdff2 = pd.DataFrame(
        #"iter": {
        #"DM": {"DM1": {
        [[0.8, 0.8, 0.8], # index on nyt step number..
        [0.7,0.7, 0.7],
        [0.6,0.6,0.6]], columns=["DM1", "DM2", "DM3"],
)
idealdff2

,DM1,DM2,DM3
0,0.8,0.8,0.8
1,0.7,0.7,0.7
2,0.6,0.6,0.6


In [117]:
pd.DataFrame(idealdff2)

,f_1,f_2,f_3
1,0.8,0.7,0.6
2,0.8,0.7,0.6
3,0.8,0.7,0.6


In [104]:
pd.DataFrame(ideal_dataframe,
    #index=[1,2,3], 
    #columns=cols
    )

,DM
DM1,"{'f_1': [0.8, 0.8, 0.8], 'f_2': [0.7, 0.7, 0.7..."


In [15]:
all_resp

[NAUTILI_Response(step_number=1, distance_to_front=33.33333333333335, reference_points={'DM1': {'f_1': 0.8, 'f_2': 0.7, 'f_3': 0.6}, 'DM2': {'f_1': 0.7, 'f_2': 0.8, 'f_3': 0.5}, 'DM3': {'f_1': 0.5, 'f_2': 0.6, 'f_3': 0.8}}, improvement_directions={'DM1': array([1.2, 1.3, 1.4]), 'DM2': array([1.3, 1.2, 1.5]), 'DM3': array([1.5, 1.4, 1.2])}, group_improvement_direction={'f_1': 1.9999999919929925, 'f_2': 1.9999999919926168, 'f_3': 1.9999999919925402}, navigation_point={'f_1': 1.5364709777317986, 'f_2': 1.5364973879863197, 'f_3': 1.5354603644147513}, reachable_solution={'f_1': 0.6094129331953962, 'f_2': 0.6094921639589597, 'f_3': 0.6063810932442545}, reachable_bounds={'lower_bounds': {'f_1': 3.90654895990358e-10, 'f_2': 0.0, 'f_3': 1.3470386024085117e-05}, 'upper_bounds': {'f_1': 1.5364709777317986, 'f_2': 1.5364973879863197, 'f_3': 1.5354603644147513}}),
 NAUTILI_Response(step_number=2, distance_to_front=66.66666666666667, reference_points={'DM1': {'f_1': 0.8, 'f_2': 0.7, 'f_3': 0.6}, 'DM

In [42]:
all_resp[0].group_improvement_direction

{'f_1': 1.9999999919929925,
 'f_2': 1.9999999919926168,
 'f_3': 1.9999999919925402}

In [50]:
dd = {"step_n":[], "Type":[], "f_1":[], "f_2":[], "f_3":[]}
for resp in all_resp:
    first_iter_gid = all_resp[0].group_improvement_direction
    for q in range(1,4): # DM määrä on q
        dd["step_n"].append(resp.step_number)
        dd["Type"].append(f"DM{q}")
        dd["f_1"].append(resp.reference_points[f"DM{q}"]["f_1"])
        dd["f_2"].append(resp.reference_points[f"DM{q}"]["f_2"])
        dd["f_3"].append(resp.reference_points[f"DM{q}"]["f_3"])
    # add navigation point
    dd["step_n"].append(resp.step_number)
    dd["Type"].append(f"NAVPOINT")
    dd["f_1"].append(resp.navigation_point["f_1"])
    dd["f_2"].append(resp.navigation_point["f_2"])
    dd["f_3"].append(resp.navigation_point["f_3"])
    # add reachable solution
    dd["step_n"].append(resp.step_number)
    dd["Type"].append(f"REACHSOL")
    dd["f_1"].append(resp.reachable_solution["f_1"])
    dd["f_2"].append(resp.reachable_solution["f_2"])
    dd["f_3"].append(resp.reachable_solution["f_3"])
    # add group improvement direction
    if resp.group_improvement_direction == None:
        dd["step_n"].append(resp.step_number)
        dd["Type"].append(f"GID")
        dd["f_1"].append(first_iter_gid["f_1"])
        dd["f_2"].append(first_iter_gid["f_2"])
        dd["f_3"].append(first_iter_gid["f_3"])
    else:
        dd["step_n"].append(resp.step_number)
        dd["Type"].append(f"GID")
        dd["f_1"].append(resp.group_improvement_direction["f_1"])
        dd["f_2"].append(resp.group_improvement_direction["f_2"])
        dd["f_3"].append(resp.group_improvement_direction["f_3"])
       # resp.reference_points["DM1"]["f_1"]
    #dd[]    , resp.navigation_point["f_1"])
    #break
dd

{'step_n': [1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3],
 'Type': ['DM1',
  'DM2',
  'DM3',
  'NAVPOINT',
  'REACHSOL',
  'GID',
  'DM1',
  'DM2',
  'DM3',
  'NAVPOINT',
  'REACHSOL',
  'GID',
  'DM1',
  'DM2',
  'DM3',
  'NAVPOINT',
  'REACHSOL',
  'GID'],
 'f_1': [0.8,
  0.7,
  0.5,
  1.5364709777317986,
  0.6094129331953962,
  1.9999999919929925,
  0.8,
  0.7,
  0.5,
  1.0729419554635975,
  0.6094129331953962,
  1.9999999919929925,
  0.8,
  0.7,
  0.5,
  0.6094129331953962,
  0.6094129331953962,
  1.9999999919929925],
 'f_2': [0.7,
  0.8,
  0.6,
  1.5364973879863197,
  0.6094921639589597,
  1.9999999919926168,
  0.7,
  0.8,
  0.6,
  1.0729947759726397,
  0.6094921639589597,
  1.9999999919926168,
  0.7,
  0.8,
  0.6,
  0.6094921639589597,
  0.6094921639589597,
  1.9999999919926168],
 'f_3': [0.6,
  0.5,
  0.8,
  1.5354603644147513,
  0.6063810932442545,
  1.9999999919925402,
  0.6,
  0.5,
  0.8,
  1.0709207288295028,
  0.6063810932442545,
  1.9999999919925402,
  0.6,
  0.5,

In [51]:
ttt = pd.DataFrame(dd)
ttt

,step_n,Type,f_1,f_2,f_3
0,1,DM1,0.800000,0.700000,0.600000
1,1,DM2,0.700000,0.800000,0.500000
2,1,DM3,0.500000,0.600000,0.800000
3,1,NAVPOINT,1.536471,1.536497,1.535460
4,1,REACHSOL,0.609413,0.609492,0.606381
5,1,GID,2.000000,2.000000,2.000000
6,2,DM1,0.800000,0.700000,0.600000
7,2,DM2,0.700000,0.800000,0.500000
8,2,DM3,0.500000,0.600000,0.800000
9,2,NAVPOINT,1.072942,1.072995,1.070921


In [20]:
objective_dict_to_numpy_array(problem, all_resp[0].reference_points["DM1"])


array([0.8, 0.7, 0.6])

In [8]:
ff

,Type,f_1,f_2,f_3,Iteration
0,DM1,0.7,0.5,0.7,0
1,DM2,0.6,0.5,0.6,0
2,DM3,0.5,0.8,0.5,0
3,NAVPOINT,1.0,1.0,1.0,0
4,GID,0.5,0.5,0.5,0
5,DM1,0.7,0.5,0.7,1
6,DM2,0.6,0.5,0.6,1
7,DM3,0.5,0.8,0.5,1
8,NAVPOINT,0.8,0.8,0.8,1
9,GID,0.5,0.5,0.5,1


In [55]:
fig4 = ex.parallel_coordinates(ttt, color="Type", labels={})
fig4

ValueError: 
    Invalid element(s) received for the 'color' property of parcoords.line
        Invalid elements include: ['DM1', 'DM2', 'DM3', 'NAVPOINT', 'REACHSOL', 'GID', 'DM1', 'DM2', 'DM3', 'NAVPOINT']

    The 'color' property is a color and may be specified as:
      - A hex string (e.g. '#ff0000')
      - An rgb/rgba string (e.g. 'rgb(255,0,0)')
      - An hsl/hsla string (e.g. 'hsl(0,100%,50%)')
      - An hsv/hsva string (e.g. 'hsv(0,100%,100%)')
      - A named CSS color:
            aliceblue, antiquewhite, aqua, aquamarine, azure,
            beige, bisque, black, blanchedalmond, blue,
            blueviolet, brown, burlywood, cadetblue,
            chartreuse, chocolate, coral, cornflowerblue,
            cornsilk, crimson, cyan, darkblue, darkcyan,
            darkgoldenrod, darkgray, darkgrey, darkgreen,
            darkkhaki, darkmagenta, darkolivegreen, darkorange,
            darkorchid, darkred, darksalmon, darkseagreen,
            darkslateblue, darkslategray, da

In [53]:
import plotly.express as ex

# näin helppoa se on kun sen vaan saa oikein !!!
fig3 = ex.scatter_3d(ttt.sort_values("step_n"),
x="f_1", y="f_2", z="f_3",
color="Type",
#range_x=[ideal[0] - 0.5, nadir[0] + 0.5],
#range_y=[ideal[1] - 0.5, nadir[1] + 0.5],
#range_z=[ideal[2] - 0.5, nadir[2] + 0.5],
animation_frame=f"step_n")


In [ ]:
def visu(problem, all_resp):
    lower_bounds = pl.DataFrame(
        [response.reachable_bounds["lower_bounds"] for response in all_resp]
    )
    upper_bounds = pl.DataFrame(
        [response.reachable_bounds["upper_bounds"] for response in all_resp]
    )
    reference_points = pl.DataFrame(
        [response.reference_points for response in all_resp[1:]]
    )
    navigation_points = pl.DataFrame(
        [response.navigation_point for response in all_resp]
    )
    reachable_sols = pl.DataFrame(
        [response.reachable_solution for response in all_resp]
    )
    group_dirs = pl.DataFrame(
        [all_resp[0].group_improvement_direction]
    )

    """
    rps = [] 
    for dm in reference_points:
        max_multiplier = [-1 if obj.maximize else 1 for obj in problem.objectives]
        rp = (
            np.array([reference_points[dm][obj.symbol] for obj in problem.objectives]) * max_multiplier
        )
        rps.append(rp)
    #rps
    """

    import plotly.express as ex

    #iteration_points = pd.DataFrame(
    #    navigation_points,
    #    columns=["f1", "f2", "f3"]
    #)
    fig2 = ex.scatter_3d(
        x=navigation_points[navigation_points.columns[0]],
        y=navigation_points[navigation_points.columns[1]],
        z=navigation_points[navigation_points.columns[2]],
    )
    # TODO: use brain to do this
    # TODO: add RPs
    # TODO: add PF?
    # TODO: add the reachable solutions solution

    #prefs = np.array(
    #    [pref[1] for pref in rps.items()]
    #)
    #prefs = np.vstack((prefs, prefs[0]))
    fig2.add_scatter3d(
        x=[0.8, 0.7, 0.6], 
        y= [0.7, 0.8, 0.5],
        z =[0.5, 0.6, 0.8],)
    #prefs = [
    #    [0.8, 0.7, 0.6],
    #    [0.7, 0.8, 0.5],
    #    [0.5, 0.6, 0.8],
    #]

    #fig2.add_scatter3d(
    #    x=prefs[:, 0],
    #    y=prefs[:, 1],
    #    z=prefs[:, 2],
    #)

    fig2.add_scatter3d(
        x=[group_dirs[0]],
        y=[group_dirs[1]],
        z=[group_dirs[2]],
    )
    fig2.show()



In [6]:
#test_dtzl2("mean")
#test_dtzl2("maxmin")
#test_dtzl2("maxmin_cones")
problem = dtlz2(10,3)
all_resp = solve_prob(problem, "maxmin")
visu(problem, all_resp)

maxmin_cones
`xtol` termination condition is satisfied.
Number of iterations: 336, function evaluations: 1680, CG iterations: 0, optimality: 1.96e-01, constraint violation: 1.00e+00, execution time: 0.27 s.
[8.00700745e-09 8.00738320e-09 8.00745991e-09]


In [4]:
ideal = get_ideal_dict(problem)
len(ideal)

3